In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import datetime
import re

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# El propósito de este notebook es únicamente recuperar el orden en que Sklearn hace el onehot encoding de algunas variables. 

In [2]:
# define categorical, numerical and date columns
CAT_COLS = ["dia_semana", "codigo_cierre", "año_cierre", "mes_cierre", "mes", "delegacion_inicio",
            "incidente_c4", "clas_con_f_alarma", "tipo_entrada", "delegacion_cierre", "hora_creacion",
           "hora_cierre"]

DATE_COLS = ["fecha_creacion", "fecha_cierre"]

NUM_COLS = ["latitud", "longitud"]#esto irá en el script ingestion .py
def ingest_file(file_name):
    """
    Function to retrieve and return the accidents dataset.
    Parameters:
    -----------
    file_name: str
               Path to the file.
    Returns:
    --------
    df: pandas dataframe
    """
    df = pd.read_csv(file_name)
    return df


def drop_cols(df):
    """
    Function to drop unnnecesary columns in the dataset.
    """
    df.drop(columns = ['folio', 'geopoint', 'mes', 'mes_cierre', 'hora_cierre', 'año_cierre'], inplace = True)
    return df



def fill_na(df):
    """
    Function to fill null values in a dataframe.
    """
    #aquí podemos ir agregando más cosas cuando descubramos 
    #cómo imputar valores faltantes para latitud y longitud
    df.fillna({
        'delegacion_inicio': 'No Disponible',
        'delegacion_cierre': 'No Disponible'
              }, inplace = True)
    return df


def categoric_transformation(col,df):
    df[col] = df[col].astype("category")
    return df 

def create_categorical(cols, df):
    """
    Function to transform and prepare the categorical features in the dataset.
    """
    #transform to appropriate data type
    for col in cols: 
        df = categoric_transformation(col, df)
     
    return df


def date_transformation(col,df):
    """
    Function to prepare and transform date-type columns. 
    """
    df[col] = pd.to_datetime(df[col], dayfirst=True)
    return df

def create_date_cols(cols, df):
    for col in cols:
        df = date_transformation(col, df)
    return df 



def generate_label(df):
    """
    Function to create a new column indicating whether there was
    a false alarm or not. 
    Parameters:
    -----------
    df: pandas dataframe
    
    Returns:
    --------
    df: pandas dataframe
    """
    #transformamos la columna para solo quedarnos con la letra del código
    df["codigo_cierre"] = df["codigo_cierre"].apply(lambda x: x[1])
    df['label'] = np.where(
        (df.codigo_cierre == 'F') | (df.codigo_cierre == 'N'), 1, 0)
    return df 


def clean_hora_creacion(df):
    """
    Function to transform hours with incorrect format to timedelta format. 
    """
    horas_raw = df.hora_creacion.values.tolist()
    horas_clean = [datetime.timedelta(days=float(e)) if e.startswith("0.") else e for e in horas_raw]
    df["hora_creacion"] = horas_clean
    return df 


def create_simple_hour(df):
    """
    Function to extract the hour from the column "hora_creacion"
    Parameters:
    -----------
    df: pandas dataframe
    
    Returns:
    ---------
    df: pandas dataframe with a new column indicating the hour. 
    """
    #la función se podria adaptar para devolver minuto o segundo pero no lo considero necesario
    pattern = '\d+' #encuentra uno o más dígitos
    horas_raw = df.hora_creacion.astype(str).values #son así: '22:35:04', '22:50:49', '09:40:11'
    n = len(horas_raw)
    horas_clean = [0]*n #es más rápido reasignar valores que hacer .append()
    for i in range(n):
        hora_raw = horas_raw[i]
        hora_clean = re.match(pattern, hora_raw)[0] #solo queremos la hora, esto devuelve un objeto
        horas_clean[i] = hora_clean
    
    df["hora_simple"] = horas_clean
    return df 


def add_date_columns(df):
    """
    Esta función es muy importante puesto que nos ayudará a crear el mes, día y año de creación
    del registro. De esta manera podemos prescindir de las fechas de cierre, que no tendríamos en tiempo
    real en un modelo. 
    Parameters:
    -----------
    df: pandas dataframe
    
    Returns:
    ---------
    df: pandas dataframe with 4 new columns
    """
    mapping_meses = {1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril", 5: "Mayo",
                       6: "Junio", 7: "Julio", 8: "Agosto", 9: "Septiembre", 10: "Octubre",
                       11: "Noviembre", 12: "Diciembre"}
    
    
    df["año_creacion"] = df.fecha_creacion.dt.year
    df["mes_creacion"] = df.fecha_creacion.dt.month
    df["dia_creacion"] = df.fecha_creacion.dt.day
    df["mes_creacion_str"] = df.mes_creacion.map(mapping_meses)
    df["año_creacion"] = df["año_creacion"].astype(str)
    return df 


def create_time_blocks(df):
    """
    Function to group the hour of the day into 3-hour blocks.
    Parameters:
    -----------
    df: pandas dataframe
    
    Returns:
    ---------
    df: pandas dataframe with a new column indicating the time-block.
    """
    horas_int = set(df.hora_simple.astype(int).values) #estaba como categórico
    f = lambda x: 12 if x == 0 else x
    mapping_hours = {}
    for hora in horas_int:
        grupo = (hora // 3) * 3
        if grupo < 12: 
            nombre_grupo = str(f(grupo)) + "-" + str(grupo + 2) + " a.m."
        else:
            hora_tarde = grupo % 12
            nombre_grupo = str(f(hora_tarde)) + "-" + str(hora_tarde + 2) + " p.m."
        mapping_hours[hora] = nombre_grupo
    
    df["espacio_del_dia"] = df["hora_simple"].astype(int).map(mapping_hours)
    return df
    

    

    
def basic_preprocessing(path):
    """
    Function to summarize all the preprocessing done to the data.
    Parameters:
    -----------
    path: str
          Path to your file
    
    Returns:
    ---------
    df: pandas dataframe
    """
    df = ingest_file(path) 
    df = generate_label(df)
    df = fill_na(df) 
    df = clean_hora_creacion(df)
    df = create_categorical(CAT_COLS, df) #transform to appropriate data types
    df = create_date_cols(DATE_COLS, df)
    df = add_date_columns(df)
    df = create_simple_hour(df)
    df = create_time_blocks(df)
    df = drop_cols(df)
    
    return df

In [3]:
# Hacemos un preprocesamiento básico
data = basic_preprocessing('incidentes-viales-c5.csv')

In [4]:
data = data.drop(['codigo_cierre',
                'fecha_creacion', 'fecha_cierre',
                'hora_creacion', 'clas_con_f_alarma',
                'dia_semana', 'mes_creacion_str',
                'delegacion_cierre'
                ], axis=1)  
#no hay que eliminar aún año creación para ordenar por eso 

# Feature Generation

In [5]:
#primero transformamos este tipo de dato
data.hora_simple = data.hora_simple.astype(int)
data.año_creacion = data.año_creacion.astype(int)

In [6]:
#añadimos columna que indique si un registro provino de llamada al 066 o 911

In [7]:
data["bool_llamada"] = np.where((data.tipo_entrada == "LLAMADA DEL 911") | 
                                (data.tipo_entrada == "LLAMADA DEL 066"), 1, 0) 

In [8]:
transformers = [('one_hot', OneHotEncoder(), ['delegacion_inicio', 'incidente_c4',
                                               'tipo_entrada', 'espacio_del_dia'])]

col_trans = ColumnTransformer(transformers, remainder="drop", n_jobs=-1)
#cambié lo de drop por passthrough para no tirar las demás

In [9]:
#importante: ordenar temporalmente el dataframe
data = data.sort_values(by=["año_creacion", "mes_creacion", "dia_creacion",
                    "hora_simple"])

In [10]:
HOURS = 24
MONTHS = 12
DAYS = 7

data['sin_hr'] = np.sin(2*np.pi*data.hora_simple/HOURS)
data['cos_hr'] = np.cos(2*np.pi*data.hora_simple/HOURS)

data['sin_month'] = np.sin(2*np.pi*data.mes_creacion/MONTHS)
data['cos_month'] = np.cos(2*np.pi*data.mes_creacion/MONTHS)

data['sin_day'] = np.sin(2*np.pi*data.dia_creacion/DAYS)
data['cos_day'] = np.cos(2*np.pi*data.dia_creacion/DAYS)

In [11]:
#OJO. 
X = col_trans.fit_transform(data.drop(columns="label"))

In [12]:
X.shape

(1383138, 60)

In [13]:
Y = data.label.values.reshape(X.shape[0],)

In [14]:
np.random.seed(1993)

In [15]:
data.head()

,delegacion_inicio,incidente_c4,latitud,longitud,tipo_entrada,label,año_creacion,mes_creacion,dia_creacion,hora_simple,espacio_del_dia,bool_llamada,sin_hr,cos_hr,sin_month,cos_month,sin_day,cos_day
1109229,BENITO JUAREZ,accidente-choque con lesionados,19.38867,-99.18856,LLAMADA DEL 066,0,2013,12,31,13,12-2 p.m.,1,-0.258819,-9.659258e-01,-2.449294e-16,1.0,0.433884,-0.900969
1109233,BENITO JUAREZ,accidente-choque con lesionados,19.38867,-99.18856,LLAMADA DEL 066,0,2013,12,31,13,12-2 p.m.,1,-0.258819,-9.659258e-01,-2.449294e-16,1.0,0.433884,-0.900969
1109239,GUSTAVO A. MADERO,lesionado-atropellado,19.49705,-99.09715,LLAMADA DEL 066,0,2013,12,31,13,12-2 p.m.,1,-0.258819,-9.659258e-01,-2.449294e-16,1.0,0.433884,-0.900969
1164423,ALVARO OBREGON,accidente-choque sin lesionados,19.37831,-99.20090,LLAMADA DEL 066,1,2013,12,31,18,6-8 p.m.,1,-1.000000,-1.836970e-16,-2.449294e-16,1.0,0.433884,-0.900969
1109062,AZCAPOTZALCO,accidente-choque con lesionados,19.48238,-99.19781,LLAMADA DEL 066,0,2013,12,31,19,6-8 p.m.,1,-0.965926,2.588190e-01,-2.449294e-16,1.0,0.433884,-0.900969


# Entrenamiento del modelo

En primer lugar tenemos que separar los datos en un conjunto de prueba y otro de entrenamiento. Dado que contamos con datos temporales, lo mejor es separar manualmente los datos usando los índices. 

In [16]:
#separación en train y test manualmente para no alterar el tiempo. 
lim = round(data.shape[0] * .010) 
X_train, X_test = X[:lim], X[lim:]
y_train, y_test = Y[:lim], Y[lim:]

In [17]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(13831, 60) (13831,)
(1369307, 60) (1369307,)


In [18]:
# ocuparemos un RF
lim = round(data.shape[0] * .010) 
X_train, X_test = X[:lim], X[lim:]
y_train, y_test = Y[:lim], Y[lim:]
classifier = RandomForestClassifier(oob_score=True, random_state=1993)
# separando en train, test

# definicion de los hiperparametros que queremos probar
hyper_param_grid = {'n_estimators': [100, 300], 
                    'max_depth': [1, 5, 10],
                    'min_samples_split': [2, 5]}

tscv = TimeSeriesSplit(n_splits=2) #añadiendo filtro por tiempo

# ocupemos grid search!
gs = GridSearchCV(classifier, 
                           hyper_param_grid, 
                           scoring = 'precision',
                           cv = tscv, 
                           n_jobs = -1)
start_time = time.time()
gs.fit(X_train, y_train)
print("Tiempo en ejecutar: ", time.time() - start_time)

Tiempo en ejecutar:  30.709311962127686


# Recuperar nombre de las columnas.

In [37]:
feature_imp = pd.Series(gs.best_estimator_.feature_importances_,index=col_trans.get_feature_names())

Los nombres de las columnas se obtuvieron como se muestra a continuación.
Podemos ver que se respeta el orden de las columnas que se pasaron al ColumnTransformer. 

In [60]:
feature_imp = pd.Series(gs.best_estimator_.feature_importances_,index=col_trans.get_feature_names())
feature_imp = pd.DataFrame(feature_imp).reset_index().rename(columns={
    "index": "valor",
    0: "importance"
})
feature_imp
#feature_imp.iloc[52:,:].valor.values

,valor,importance
0,one_hot__x0_ALVARO OBREGON,0.00
1,one_hot__x0_AZCAPOTZALCO,0.01
2,one_hot__x0_BENITO JUAREZ,0.01
3,one_hot__x0_COYOACAN,0.06
4,one_hot__x0_CUAJIMALPA,0.01
5,one_hot__x0_CUAUHTEMOC,0.09
6,one_hot__x0_GUSTAVO A. MADERO,0.02
7,one_hot__x0_IZTACALCO,0.00
8,one_hot__x0_IZTAPALAPA,0.01
9,one_hot__x0_MAGDALENA CONTRERAS,0.01


In [25]:
data.head()

,delegacion_inicio,incidente_c4,latitud,longitud,tipo_entrada,label,año_creacion,mes_creacion,dia_creacion,hora_simple,espacio_del_dia,bool_llamada,sin_hr,cos_hr,sin_month,cos_month,sin_day,cos_day
1109229,BENITO JUAREZ,accidente-choque con lesionados,19.38867,-99.18856,LLAMADA DEL 066,0,2013,12,31,13,12-2 p.m.,1,-0.258819,-9.659258e-01,-2.449294e-16,1.0,0.433884,-0.900969
1109233,BENITO JUAREZ,accidente-choque con lesionados,19.38867,-99.18856,LLAMADA DEL 066,0,2013,12,31,13,12-2 p.m.,1,-0.258819,-9.659258e-01,-2.449294e-16,1.0,0.433884,-0.900969
1109239,GUSTAVO A. MADERO,lesionado-atropellado,19.49705,-99.09715,LLAMADA DEL 066,0,2013,12,31,13,12-2 p.m.,1,-0.258819,-9.659258e-01,-2.449294e-16,1.0,0.433884,-0.900969
1164423,ALVARO OBREGON,accidente-choque sin lesionados,19.37831,-99.20090,LLAMADA DEL 066,1,2013,12,31,18,6-8 p.m.,1,-1.000000,-1.836970e-16,-2.449294e-16,1.0,0.433884,-0.900969
1109062,AZCAPOTZALCO,accidente-choque con lesionados,19.48238,-99.19781,LLAMADA DEL 066,0,2013,12,31,19,6-8 p.m.,1,-0.965926,2.588190e-01,-2.449294e-16,1.0,0.433884,-0.900969
